In [1]:
import pandas as pd
import joblib
import numpy as np

# Loading data

In [2]:
X = joblib.load("Inputs")
X.shape

(8732, 40)

In [3]:
Y = joblib.load("Outputs")
Y.shape

(8732,)

# Encoding data

In [4]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [5]:
Y = LE.fit_transform(Y)
Y

array([3, 2, 2, ..., 1, 1, 1], dtype=int64)

# Splitting  data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=2)

In [7]:
X_train.shape

(6286, 40)

# Model Building

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
model=Sequential()

###first layer
model.add(Dense(256,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###second layer
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))

###third layer
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))

###final layer
model.add(Dense(len(set(Y))))
model.add(Activation('softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               10496     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 activation_1 (Activation)   (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [11]:
# compiling the model
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [12]:
# EarlyStopping to save compute resources
ES = EarlyStopping(monitor='val_loss',mode='min',patience=8)

In [13]:
# fitting the model
model.fit(X_train, Y_train, batch_size = 32, epochs = 200, validation_data=(X_val,Y_val), callbacks=ES)

Epoch 1/200
197/197 [==============================] - 1s 3ms/step - loss: 7.1583 - accuracy: 0.1269 - val_loss: 2.2584 - val_accuracy: 0.1617
Epoch 2/200
197/197 [==============================] - 0s 2ms/step - loss: 2.4731 - accuracy: 0.1492 - val_loss: 2.2359 - val_accuracy: 0.1803
Epoch 3/200
197/197 [==============================] - 0s 2ms/step - loss: 2.2477 - accuracy: 0.1806 - val_loss: 2.0936 - val_accuracy: 0.2589
Epoch 4/200
197/197 [==============================] - 0s 2ms/step - loss: 2.1362 - accuracy: 0.2043 - val_loss: 1.9843 - val_accuracy: 0.2532
Epoch 5/200
197/197 [==============================] - 0s 2ms/step - loss: 2.0437 - accuracy: 0.2485 - val_loss: 1.9425 - val_accuracy: 0.2876
Epoch 6/200
197/197 [==============================] - 0s 2ms/step - loss: 1.9789 - accuracy: 0.2704 - val_loss: 1.8452 - val_accuracy: 0.3534
Epoch 7/200
197/197 [==============================] - 0s 2ms/step - loss: 1.8958 - accuracy: 0.2911 - val_loss: 1.7194 - val_accuracy: 0.4192

Epoch 58/200
197/197 [==============================] - 0s 2ms/step - loss: 0.7313 - accuracy: 0.7549 - val_loss: 0.5847 - val_accuracy: 0.7997
Epoch 59/200
197/197 [==============================] - 0s 2ms/step - loss: 0.7277 - accuracy: 0.7585 - val_loss: 0.5690 - val_accuracy: 0.8269
Epoch 60/200
197/197 [==============================] - 0s 2ms/step - loss: 0.7095 - accuracy: 0.7588 - val_loss: 0.5595 - val_accuracy: 0.8140
Epoch 61/200
197/197 [==============================] - 0s 2ms/step - loss: 0.7085 - accuracy: 0.7588 - val_loss: 0.5717 - val_accuracy: 0.8155
Epoch 62/200
197/197 [==============================] - 0s 2ms/step - loss: 0.7140 - accuracy: 0.7593 - val_loss: 0.5477 - val_accuracy: 0.8083
Epoch 63/200
197/197 [==============================] - 0s 2ms/step - loss: 0.7032 - accuracy: 0.7660 - val_loss: 0.5537 - val_accuracy: 0.8197
Epoch 64/200
197/197 [==============================] - 0s 2ms/step - loss: 0.6973 - accuracy: 0.7689 - val_loss: 0.5464 - val_accuracy:

# Evaluation

In [14]:
test_accuracy=model.evaluate(X_test,Y_test,verbose=0)
print(test_accuracy[1])

0.8277046084403992


In [15]:
Y_pred = model.predict(X_test)
Y_pred_argmax = np.argmax(Y_pred, axis=1)


55/55 [==============================] - 0s 1ms/step


In [16]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_pred_argmax))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       203
           1       0.96      0.82      0.88        89
           2       0.59      0.80      0.67       176
           3       0.93      0.69      0.79       241
           4       0.93      0.83      0.88       206
           5       0.94      0.88      0.91       227
           6       0.79      0.56      0.66        82
           7       0.88      0.97      0.92       189
           8       0.91      0.92      0.91       168
           9       0.70      0.75      0.72       166

    accuracy                           0.83      1747
   macro avg       0.84      0.81      0.82      1747
weighted avg       0.84      0.83      0.83      1747



# Dumping Model

In [17]:
joblib.dump(model,"model_85acc")

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\activation
......vars
...layers\activation_1
......vars
...layers\activation_2
......vars
...layers\activation_3
......vars
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-08-03 18

['model_85acc']

# Testing

In [18]:
import librosa
def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

In [19]:
def get_my_predictions(filename):
    prediction_feature=get_features(filename)
    prediction_feature=prediction_feature.reshape(1,-1)
    op = model.predict(prediction_feature)
    return LE.inverse_transform([np.argmax(op)])

In [20]:
pd.Series(LE.classes_)

0     air_conditioner
1            car_horn
2    children_playing
3            dog_bark
4            drilling
5       engine_idling
6            gun_shot
7          jackhammer
8               siren
9        street_music
dtype: object

In [29]:
# Dog Bark
print(get_my_predictions(r"C:\Users\Admin\Downloads\dog-barking-70772.mp3"))
print(get_my_predictions(r"C:\Users\Admin\Downloads\barking-156375.mp3"))

1/1 [==============================] - 0s 16ms/step
['dog_bark']
1/1 [==============================] - 0s 16ms/step
['children_playing']


In [30]:
# Jackhammer
print(get_my_predictions(r"C:\Users\Admin\Downloads\construction_site-19522.mp3"))
print(get_my_predictions(r"C:\Users\Admin\Downloads\jackhammer-01-62270.mp3"))

1/1 [==============================] - 0s 16ms/step
['jackhammer']
1/1 [==============================] - 0s 17ms/step
['street_music']


In [31]:
# carhorn
print(get_my_predictions(r"C:\Users\Admin\Downloads\car-horn-6408.mp3"))
print(get_my_predictions(r"C:\Users\Admin\Downloads\car-horn-beep-beep-two-beeps-honk-honk-6188.mp3"))

1/1 [==============================] - 0s 17ms/step
['children_playing']
1/1 [==============================] - 0s 17ms/step
['car_horn']
